# Police Patrolling Route Optimization

## Background
Aiming to enforce the law and to ensure the safety, police services is an essential part of modern countries. One of the services is police patrolling, providing protection of property and lives as well as prevention and detection of crime. As of Toronto, a city with over 6.3 million population, police patrolling is of critical necessity. 
However, The breakout of Covid leads to increasing spending on healthcare and decreasing budget on other governmental services, Officials are constantly facing the problem of balancing between fund spending and high quality of services. Therefore, decision making starts to play a more and more significant role for police services. 

## Problem Description

Considering the limitations on budget, patrolling time, and police resources; as well as the needs to reinforce patrolling services in regions with high crime rates, we brought up this question: How should the police offices of Toronto dispatch the police cars, and which patrolling routes should the cars follow? To answer the question, we aim to provide an minimization of  the budget spent on police car patrolling among police office divisions in downtown Toronto via optimizing the patrolling routes.<br><br>
We select 8 police divisions from the City of Toronto in the downtown areas. Our patrol route begin with dispatching 3 police cars from Central Field--Division 53 to other divisions. We assume that there is a 1h time limit on police patrol. Since police patrol has a cost of 37.38 dollars per hour, we then need to minimize the budget while subject to time constraint.

#  Base Model Formulation


### Sets and Indices

$i,j \in \text{Divisions} \equiv D=\{0,1..(n-1)\}$: Set of divisions where $0$ is the index for the Central Field, and $n$ is the number of divisions.

$k \in \text{Police cars} \equiv  P=\{0..K-1\}$: Index and set of police cars, where $K$ is the number of police cars.

$S_k \in S  $: Tour of police car $k$, i.e. subset of divisions visited by the police car.

### Parameters

$t_{i,j} \in \mathbb{R}^+$: Travel time from division $i$  to division $j$.

### Decision Variables

$x_{i,j,k} \in \{0,1 \}$: This binary variable is equal 1, if police car $k$ visits and goes directly from division $i$ to division $j$, and zero otherwise.

$y_{i,k} \in \{0,1 \}$: This binary variable is equal 1, if police car $k$ visits division $i$, and zero otherwise.

$z_{k} \in \{0,1 \}$: This binary variable is equal 1, if police car $k \in \{1,2..K\}$ is used, and zero otherwise.

$t_{k} \in \mathbb{R}^{+}$: This continuous variable denotes the travel time for police car $k \in \{1,2..K\}$.

$s \in \mathbb{R}^{+}$: This continuous variable denotes $max(t_{k})\:\forall$ police car $k \in \{1,2..K\}$.

$st \in \mathbb{R}^{+}$: This continuous variable is the sum of travel time for police car $k \in \{1,2..K\}$. We use it to calculate the total patrolling cost.

### Objective Functions

**First priority:**: Minimize the total travel time, which is the sum of travel time for each police car.

\begin{equation}
\text{Minimize} \quad \sum_{k = 1}^{K} t_k
\end{equation}

**Second priority:**: Minimize the maximum travel time, which is the time for all police cars to complete patrol.

\begin{equation}
\text{Minimize} \quad max(t_{k})
\end{equation}

**Third priority:**: Minimize the number of police cars used.

\begin{equation}
\text{Minimize} \quad \sum_{k = 1}^{K} z_k
\end{equation}

### Constraints

**Police car utilization**: For all divisions different from the Central Field, i.e. $i > 0$, if the division is visited by a police car $k$, then it is used.

\begin{equation}
y_{i,k} \leq z_{k} \quad \forall i \in D ,\:i>0, \; k \in P
\end{equation}

**Travel time**: No police car travels for more than 60 min. Note that we do not consider the travel time to return to Central Field.

\begin{equation}
\sum_{i \in D} \sum_{j \in D ,j>0} t_{i,j} \cdot x_{i,j,k} \leq 120 \quad \forall k \in  P
\end{equation}

**Visit all divisions**:  Each division is visited by exactly one police car.

\begin{equation}
\sum_{k \in P}  y_{i,k} = 1 \quad \forall i \in D ,\:i>0
\end{equation}

**Starting point**: Central Field is visited by every police car used. 

\begin{equation}
\sum_{k \in P}  y_{1,k} \geq \sum_{k \in P} z_k
\end{equation}

**Arriving at a division**: If division $j$ is visited by police car $k$, then the police car is coming from another division $i$.

\begin{equation}
\sum_{i \in D}  x_{i,j,k} =  y_{j,k}  \quad \forall j \in D, \; k \in P
\end{equation}

**Leaving a division**: If a police car $k$ leaves division $j$, then the police car is going to another division $i$.

\begin{equation}
\sum_{i \in D}  x_{j,i,k} = y_{j,k}  \quad \forall j \in D, \; k \in P
\end{equation}

**Breaking symmetry**: all routes that visit the same divisions are only considered as one route

\begin{equation}
\sum_{i \in D}  y_{i,k} \geq \sum_{i \in D}  y_{i,k+1} \quad \forall k \in  \{0..K-1\}
\end{equation}

**Subtour elimination**: These constraints ensure that for each police car route, there is no cycle. 

\begin{equation}
\sum_{(i,j) \in S_k}x_{i,j,k} \leq |S_k|-1 \quad \forall  k \in K, \;   S_k \subseteq D
\end{equation}



## Python Implementation

In [1]:
import sys
import math
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from itertools import permutations

## Input data  

In [2]:
#import dataset for travel time between each division
df = pd.read_excel('police travel time.xlsx')
df = df.iloc[1:12,2:13]
df_array = df.to_numpy()

# number of divisions, including Central Field. The index of Central Field is 0
n = 8
divisions = [*range(n)]

# number of police cars
K = 3
cars = [*range(K)]

# travel time between each division
time = {(i, j):df_array[i,j] for i in range(n) for j in range(n) if i != j}

#cost per minute for the patrol
cost = 37.38/60 

## Model Deployment

We create our base model and the variables. The decision variables determines the order in which each police car visits a subset of divisions, which division is visited by each police car, and if a police car is used or not.

In [3]:
m = gp.Model('toronto_police_patrol.lp')

# Create variables: 

# x = 1, if police car k visits and goes directly from division i to division j 
x = m.addVars(time.keys(), cars, vtype=GRB.BINARY, name='FromToBy')

# y = 1, if division i is visited by police car k
y = m.addVars(divisions, cars, vtype=GRB.BINARY, name='visitBy')

# Number of police cars used is a decision variable
z = m.addVars(cars, vtype=GRB.BINARY, name='used')

# Travel time per police car
t = m.addVars(cars, ub=60, name='travelTime')

# Maximum travel time
s = m.addVar(name='maxTravelTime')

# Total travel time
st = m.addVar(name='TotalTravelTime')

Academic license - for non-commercial use only - expires 2022-09-25
Using license file C:\Users\DELL\gurobi.lic


## Constraints

For all locations different from Central Field, i.e. $i > 0$, if the division is visited by police car $k$, then it is used.

In [4]:
# Police car utilization constraint

visitDivision = m.addConstrs((y[i,k] <= z[k]  for k in cars for i in divisions if i > 0), name='visitDivision' )

No police car travels for more than 60 min. 

In [5]:
# Travel time constraint
# Exclude the time to return to Central Field

travelTime = m.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) == t[k] for k in cars), 
                          name='travelTimeConstr' )

Each division is visited by exactly one police car.

In [6]:
# Visit all divisions

visitAll = m.addConstrs((y.sum(i,'*') == 1 for i in divisions if i>0), name='visitAll' )

Division 12, 14, 51 should be patrolled twice, but with different police cars.

Central Field is visited by every police car used.

In [7]:
# Central Field constraint

centralField = m.addConstr(y.sum(0,'*') >= z.sum(), name='centralField' )

If division  j  is visited by police car  k , then the police car is coming from another division  i.

In [8]:
# Arriving at a division constraint

ArriveConstr = m.addConstrs((x.sum('*',j,k) == y[j,k] for j,k in y.keys()), name='ArriveConstr' )

 If police car  k  leaves division  j , then the police car is going to another division  i.

In [9]:
# Leaving a division constraint

LeaveConstr = m.addConstrs((x.sum(j,'*',k) == y[j,k] for j,k in y.keys()), name='LeaveConstr' )

Breaking symmetry constraints.

In [10]:
breakSymm = m.addConstrs((y.sum('*',k-1) >= y.sum('*',k) for k in cars if k>0), name='breakSymm' )

Relate the maximum travel time to the travel times of each police car

In [11]:
maxTravelTime = m.addConstrs((t[k] <= s for k in cars), name='maxTravelTimeConstr')

Relate the Total travel time to the travel times of each police car

In [12]:
maxTravelTime = m.addConstr((t.sum('*') == st), name='TotalTravelTimeConstr')

### Objective Function

In [13]:
m.ModelSense = GRB.MINIMIZE

# First priority: Minimize the total travel time
m.setObjectiveN(st, 2, priority=0, name="Total Travel time")

# Second priority: Minimize the maximum travel time
m.setObjectiveN(s, 1, priority=1, name="Maximum Travel time")

# Third priority: Minimize the number of police cars used
m.setObjectiveN(z.sum(), 0, priority=2, name="Number of cars")

### Callback Definition
Subtour constraints prevent a police car from visiting a set of destinations without starting or ending at Central Field. Because there are an exponential number of these constraints, we don't want to add them all to the model. Instead, we use a callback function to find violated subtour constraints and add them to the model as lazy constraints.

In [14]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i,j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n: 
            for k in cars:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour not containing Central Field (0)
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # initial length has 1 more division
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

## Solve the model

In [15]:
# Run optimization engine
m.Params.LogToConsole=0
m._x = x
m.Params.LazyConstraints = 1
m.optimize(subtourelim)

## Analysis

The optimal route of each police car used and the total patrol time report follows.

In [16]:
print('Police Division index:')
print('1: Central Field, 75 Eglinton Av. W.\n2: 11 Division, 2054 Davenport Rd.\n3: 12 Division, 200 Trethewey Dr.')
print('4: 13 Division, 1435 Eglinton Av. W.\n5: 14 Division, 350 Dovercourt Rd. \n6: 51 Division, 51 Parliament St.')
print('7: 52 Division, 255 Dundas St. W.\n8: 55 Division, 101 Coxwell Av.')
print(' ')
print('Optimization Results:')

for k in cars:
    route = gp.tuplelist((i,j) for i,j in time.keys() if x[i,j,k].X > 0.5)
    if route:
        i = 0
        print(f"Route for police car {k+1}: {i+1}", end='')
        while True:
            i = route.select(i, '*')[0][1]
            if i == 8:
                print(f" -> {2+1}", end='')
            elif i == 9:
                print(f" -> {4+1}", end='')
            elif i == 10:
                print(f" -> {5+1}", end='')
            else:
                print(f" -> {i+1}", end='')
            if i == 0:
                break
        print(f". \nTravel time: {round(t[k].X,2)} min")

print(f"Total travel cost: $ {round(st.X,2)*cost}")     
print(f"Max travel time: {round(s.X,2)}"+' min')

Police Division index:
1: Central Field, 75 Eglinton Av. W.
2: 11 Division, 2054 Davenport Rd.
3: 12 Division, 200 Trethewey Dr.
4: 13 Division, 1435 Eglinton Av. W.
5: 14 Division, 350 Dovercourt Rd. 
6: 51 Division, 51 Parliament St.
7: 52 Division, 255 Dundas St. W.
8: 55 Division, 101 Coxwell Av.
 
Optimization Results:
Route for police car 1: 1 -> 4 -> 3 -> 2 -> 5 -> 1. 
Travel time: 52.0 min
Route for police car 2: 1 -> 7 -> 6 -> 8 -> 1. 
Travel time: 41.0 min
Total travel cost: $ 57.939
Max travel time: 52.0 min


# Problem Extension
Thinking beyond our base model: we must now consider that in cities, certain parts are “safer” than others, and Toronto is no exception. Therefore, it would be logical to increase police car patrols in those areas where more crime is occuring. Using Toronto Homicides dataset from 2004 to 2021, we notice 3 divisions with higher homicide rates: D12, D14, D51. Hence, we set up a new problem where we now visit the high concentration crime areas twice.

### New Set and Indices

$v \in \left \{\text{Division 12, Division 14, Division 51}\right \} \subseteq D= \{2,4,5\}$: Set of divisions that need to patrolled twice, <br>$2$ are the indexes for Division 12, $4$ are the indexes for Division 14, and $5$ are the indexes for Division 51.<br><br>
$w \in \left \{\text{Division 12, Division 14, Division 51}\right \} \equiv ExtraD=\{8,9,10\}$: Set of divisions that need to patrolled twice,<br>$8$ are the indexes for Division 12, $9$ are the indexes for Division 14, and $10$ are the indexes for Division 51.<br><br>
$d \in D$: a general denotion of all divisions.

### New Constraints

**Forward-Backward Prevention**: prevent police car from first patrol a high crime rate area, then patrol another area, then goes back to the same high crime rate area.

$ \forall\:d\in D, d \notin \{v,w\}, d>0, k\in K$:
\begin{equation}
x_{v,d,k}+x_{d,w,k}\leq1,\: v=2, w=8
\end{equation}
\begin{equation}
x_{v,d,k}+x_{d,w,k}\leq1,\: v=4, w=9
\end{equation}
\begin{equation}
x_{v,d,k}+x_{d,w,k}\leq1,\: v=5, w=10
\end{equation}
\begin{equation}
x_{w,d,k}+x_{d,v,k}\leq1,\: v=2, w=8
\end{equation}
\begin{equation}
x_{w,d,k}+x_{d,v,k}\leq1,\: v=4, w=9
\end{equation}
\begin{equation}
x_{w,d,k}+x_{d,v,k}\leq1,\: v=5, w=10
\end{equation}

## Input Data

In [17]:
#import dataset for travel time between each division
df = pd.read_excel('police travel time - extension.xlsx')
df = df.iloc[1:12,2:13]
df_array = df.to_numpy()

# number of divisions, including Central Field. The index of Central Field is 0
n=11
divisions = [*range(n)]

# number of police cars
K = 3
cars = [*range(K)]

# travel time between each division
time = {(i, j):df_array[i,j] for i in range(n) for j in range(n) if i != j}

#cost per minute for the patrol
cost = 37.38/60 

# D12,14,51 is high crime rate area, requires 2 patrols (2,4,5)

## Model Deployment

In [18]:
m = gp.Model('toronto_police_patrol.lp')

# Create variables: 

# x = 1, if police car k visits and goes directly from division i to division j 
x = m.addVars(time.keys(), cars, vtype=GRB.BINARY, name='FromToBy')

# y = 1, if division i is visited by police car k
y = m.addVars(divisions, cars, vtype=GRB.BINARY, name='visitBy')

# Number of police cars used is a decision variable
z = m.addVars(cars, vtype=GRB.BINARY, name='used')

# Travel time per police car
t = m.addVars(cars, ub=60, name='travelTime')

# Maximum travel time
s = m.addVar(name='maxTravelTime')

# Total travel time
st = m.addVar(name='TotalTravelTime')

## Constraints

For all locations different from Central Field, i.e. $i > 0$, if the division is visited by police car $k$, then it is used.

In [19]:
# Police car utilization constraint

visitDivision = m.addConstrs((y[i,k] <= z[k]  for k in cars for i in divisions if i > 0), name='visitDivision' )

No police car travels for more than 60 min.

In [20]:
# Travel time constraint
# Exclude the time to return to Central Field

travelTime = m.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) == t[k] for k in cars), 
                          name='travelTimeConstr' )

Each division is visited by exactly one police car.

In [21]:
# Visit all divisions

visitAll = m.addConstrs((y.sum(i,'*') == 1 for i in divisions if i>0), name='visitAll' )

Division 12, 14, 51 should be patrolled twice, but with different police cars.

In [22]:
#prevent going forward and going back for division 12, 14, 51

different_visit1 = m.addConstrs((x[2,d,k]+x[d,8,k]<=1 for d in divisions for k in cars if (d !=8)&(d!=2)&(d>0)),name='different_visit')
different_visit2 = m.addConstrs((x[8,d,k]+x[d,2,k]<=1 for d in divisions for k in cars if (d !=8)&(d!=2)&(d>0)),name='different_visit1')
different_visit3 = m.addConstrs((x[4,d,k]+x[d,9,k]<=1 for d in divisions for k in cars if (d !=4)&(d!=9)&(d>0)),name='different_visit2')
different_visit4 = m.addConstrs((x[9,d,k]+x[d,4,k]<=1 for d in divisions for k in cars if (d !=4)&(d!=9)&(d>0)),name='different_visit3')
different_visit5 = m.addConstrs((x[5,d,k]+x[d,10,k]<=1 for d in divisions for k in cars if (d !=5)&(d!=10)&(d>0)),name='different_visit4')
different_visit6 = m.addConstrs((x[10,d,k]+x[d,5,k]<=1 for d in divisions for k in cars if (d !=5)&(d!=10)&(d>0)),name='different_visit5')

Central Field is visited by every police car used.

In [23]:
# Central Field constraint

centralField = m.addConstr(y.sum(0,'*') >= z.sum(), name='centralField' )

If division  j  is visited by police car  k , then the police car is coming from another division  i.

In [24]:
# Arriving at a division constraint

ArriveConstr = m.addConstrs((x.sum('*',j,k) == y[j,k] for j,k in y.keys()), name='ArriveConstr' )

 If police car  k  leaves division  j , then the police car is going to another division  i.

In [25]:
# Leaving a division constraint

LeaveConstr = m.addConstrs((x.sum(j,'*',k) == y[j,k] for j,k in y.keys()), name='LeaveConstr' )

Breaking symmetry constraints.

In [26]:
breakSymm = m.addConstrs((y.sum('*',k-1) >= y.sum('*',k) for k in cars if k>0), name='breakSymm' )

Relate the maximum travel time to the travel times of each police car

In [27]:
maxTravelTime = m.addConstrs((t[k] <= s for k in cars), name='maxTravelTimeConstr')

Relate the Total travel time to the travel times of each police car

In [28]:
maxTravelTime = m.addConstr((t.sum('*') == st), name='TotalTravelTimeConstr')

### Objective Function

In [29]:
m.ModelSense = GRB.MINIMIZE

# First priority: Minimize the total travel time
m.setObjectiveN(st, 2, priority=0, name="Total Travel time")

# Second priority: Minimize the maximum travel time
m.setObjectiveN(s, 1, priority=1, name="Maximum Travel time")

# Third priority: Minimize the number of police cars used
m.setObjectiveN(z.sum(), 0, priority=2, name="Number of cars")

### Callback Definition
Subtour constraints prevent a police car from visiting a set of destinations without starting or ending at Central Field. Because there are an exponential number of these constraints, we don't want to add them all to the model. Instead, we use a callback function to find violated subtour constraints and add them to the model as lazy constraints.

In [30]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i,j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n: 
            for k in cars:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour not containing Central Field (0)
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # initial length has 1 more division
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

## Solve the model

In [31]:
# Run optimization engine
m.Params.LogToConsole=0
m._x = x
m.Params.LazyConstraints = 1
m.optimize(subtourelim)

## Analysis

The optimal route of each police car used and the total patrol time report follows.

In [32]:
print('Police Division index:')
print('1: Central Field, 75 Eglinton Av. W.\n2: 11 Division, 2054 Davenport Rd.\n3: 12 Division, 200 Trethewey Dr.')
print('4: 13 Division, 1435 Eglinton Av. W.\n5: 14 Division, 350 Dovercourt Rd. \n6: 51 Division, 51 Parliament St.')
print('7: 52 Division, 255 Dundas St. W.\n8: 55 Division, 101 Coxwell Av.')
print(' ')
print('Optimization Results:')

for k in cars:
    route = gp.tuplelist((i,j) for i,j in time.keys() if x[i,j,k].X > 0.5)
    if route:
        i = 0
        print(f"Route for police car {k+1}: {i+1}", end='')
        while True:
            i = route.select(i, '*')[0][1]
            if i == 8:
                print(f" -> {2+1}", end='')
            elif i == 9:
                print(f" -> {4+1}", end='')
            elif i == 10:
                print(f" -> {5+1}", end='')
            else:
                print(f" -> {i+1}", end='')
            if i == 0:
                break
        print(f". \nTravel time: {round(t[k].X,2)} min")

print(f"Total travel cost: $ {round(st.X,2)*cost}")     
print(f"Max travel time: {round(s.X,2)}"+' min')

Police Division index:
1: Central Field, 75 Eglinton Av. W.
2: 11 Division, 2054 Davenport Rd.
3: 12 Division, 200 Trethewey Dr.
4: 13 Division, 1435 Eglinton Av. W.
5: 14 Division, 350 Dovercourt Rd. 
6: 51 Division, 51 Parliament St.
7: 52 Division, 255 Dundas St. W.
8: 55 Division, 101 Coxwell Av.
 
Optimization Results:
Route for police car 1: 1 -> 6 -> 8 -> 7 -> 6 -> 1. 
Travel time: 57.0 min
Route for police car 2: 1 -> 4 -> 3 -> 5 -> 1. 
Travel time: 51.0 min
Route for police car 3: 1 -> 3 -> 2 -> 5 -> 1. 
Travel time: 49.0 min
Total travel cost: $ 97.81099999999999
Max travel time: 57.0 min
